In [2]:
from lang_utils import *
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

/opt/conda/envs/funes-pixi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lm = get_llm("llama3", "TGI")


2024-06-13 18:00:32.558 
  command:

    streamlit run /opt/conda/envs/funes-pixi/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
example_qa = dspy.Example(question="Which is the region whose capital is Zaragoza?", answer= "Aragon") #.with_inputs("question")

# input_key_only = article_summary.inputs()
# non_input_key_only = article_summary.labels()
example_qa


Example({'question': 'Which is the region whose capital is Zaragoza?', 'answer': 'Aragon'}) (input_keys=None)

In [ ]:
print("Example object with Input fields only:", input_key_only)
print("Example object with Non-Input fields only:", non_input_key_only)

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Define the predictor.
predictor = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = predictor(question=example_qa.question)

# Print the input and the prediction.
print(f"Question: {example_qa.question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Actual Answer: {example_qa.answer}")

In [ ]:
len(lm.history)

In [ ]:
lm.inspect_history(n=1)

In [5]:
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=lm, rm=colbert)

In [8]:
# Load data
#train_size=200, dev_size=300
train_size=200
dev_size=200
test_size=300
dataset = HotPotQA(train_seed=1, train_size=train_size, eval_seed=2023, dev_size=dev_size, test_size=0)
print(dataset)


/opt/conda/envs/funes-pixi/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [7]:
len(dataset.test)

0

In [ ]:

# trainset = [x.with_inputs('question') for x in dataset.train[0:150]]
# valset = [x.with_inputs('question') for x in dataset.train[150:200]]
# devset = [x.with_inputs('question') for x in dataset.dev[0:10]]

# # show an example datapoint; it's just a question-answer pair
# trainset[0]


In [7]:
# Question
question=dataset.dev[10]['question']
answer=dataset.dev[10]['answer']
print(f"Q: {question}\nA: {answer}")

Q: What year did the mountain known in Italian as "Monte Vesuvio", erupt?
A: 79 AD


In [ ]:
devset

In [8]:
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

In [9]:
config = dict(num_threads=8, display_progress=True, display_table=5)
evaluate = Evaluate(devset=devset, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


,question,example_answer,gold_titles,observations,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","[['Hebei | Hebei (; postal: Hopeh) is a province of China in the North China region. Its one-character abbreviation is ""冀 "" (Jì), named after...","No, Cangzhou is in Hebei province, but Qionghai is in Hainan province",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[['Query (complexity) | In descriptive complexity, a query is a mapping from structures of one signature to structures of another vocabulary. Neil Immerman, in his...",answer,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","[['Query (complexity) | In descriptive complexity, a query is a mapping from structures of one signature to structures of another vocabulary. Neil Immerman, in his...",,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[['Query (complexity) | In descriptive complexity, a query is a mapping from structures of one signature to structures of another vocabulary. Neil Immerman, in his...",answer,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[['Æthelweard | Æthelweard (also Ethelweard, Aethelweard, Athelweard, et cetera) is an Anglo-Saxon male name. It might refer to:'], ['Ealhswith | Ealhswith or Ealswitha (died 5...",“King Alfred the Great”,False


10.0

In [ ]:
lm.inspect_history(n=10)

In [ ]:
config = dict(max_bootstrapped_demos=2, max_labeled_demos=0, num_candidate_programs=5, num_threads=8)
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **config)
optimized_react = tp.compile(agent, trainset=trainset, valset=valset)

In [ ]:
optimized_react.save("optimized_react.json")

In [ ]:
loaded_optimized_react = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])
loaded_optimized_react.load("file.json")